In [117]:
import sys
sys.path.append('./../../src')

# python's shit
from os import path
from datetime import date, datetime, timedelta
import time
import calendar

# someone elses shit
import finnhub
import pandas

# my shit
import config
from lib.stonk_jar import StonkJar

In [118]:
finnhub_client = finnhub.Client(api_key=config.api_keys['finnhub']['live'])
ticker = 'GME'
jar = StonkJar(ticker)

In [119]:
# Time junk, just for testing?
now = time.time()
today = date.today()
yesterday = today + timedelta(days = -1)
time_period = 'D'

In [120]:
def recommendation_trends_by_date(ticker, date):
    first_of_month = date.replace(day = 1)
    recommendations = jar.pickle_back(
        "{0}_recommendations.pkl".format(ticker),
        finnhub_client.recommendation_trends,
        ticker)
    recommendation = [r for r in recommendations if datetime.strptime(r['period'], '%Y-%m-%d').date() == first_of_month]
    if (len(recommendation) > 0):
        return recommendation[0]
    return {}

In [121]:
def company_earnings_by_date(ticker, date):
    reporting_period = timedelta(days = 90)
    earnings = jar.pickle_back(
        "{0}_earnings.pkl".format(ticker),
        finnhub_client.company_earnings,
        ticker)
    earning = [r for r in earnings if datetime.strptime(r['period'], '%Y-%m-%d').date() + reporting_period > date]
    if (len(earning) > 0):
        return earning[0]
    return {}

Build a base data frame from the stock's low, open, close, high and volume for the given time period

In [122]:
def stock_candles_by_date(ticker, date):
    first_of_month_ts = int(datetime.combine(date.replace(day = 1), datetime.min.time()).timestamp())
    next_month = date.replace(day = 28) + timedelta(days = 4)
    last_of_month = next_month - timedelta(days = next_month.day)
    last_of_month_ts = datetime.combine(last_of_month, datetime.max.time()).timestamp()
    earliest = int(min(time.time(), last_of_month_ts))
    candles = jar.pickle_back(
        "{0}_candles.{1}.{2}.pkl".format(ticker, time_period, datetime.strftime(date, '%Y-%m-%d')),
        finnhub_client.stock_candles,
        'AAPL', time_period, first_of_month_ts, earliest)
    return list(zip(candles['t'], candles['l'], candles['o'], candles['c'], candles['h'], candles['v']))

Construct a set of data frames that each hold:
- timestamp
- low
- open
- close
- high
- volume
- eps_actual (the actual EPS of the last relevant reporting period of the data frame's time period)
- eps_estimate (the estimated EPS, same as above)
- rec_strong_sell
- rec_sell
- rec_hold
- rec_buy
- rec_strong_sell (this and the above 4 fields are proportions of each rec cat of the total [0 - 1])

In [123]:
def construct_data_frames(ticker, date):
    trend = recommendation_trends_by_date(ticker, date)
    trend = { **trend, **{
        'strongSell': 0,
        'sell': 0,
        'hold': 0,
        'buy': 0,
        'strongBuy': 0
    } }
    trend_total = trend['strongSell'] + trend['sell'] + trend['hold'] + trend['buy'] + trend['strongBuy']
    trend_total = 1 if trend_total == 0 else trend_total
    earnings = company_earnings_by_date(ticker, date)
    static_data = []
    if 'actual' in earnings and 'estimate' in earnings:
        static_data = static_data + [earnings['actual'], earnings['estimate']]
    else:
        static_data = static_data + [0, 0]
    static_data = static_data + [
        trend['strongSell'] / trend_total,
        trend['sell'] / trend_total,
        trend['hold'] / trend_total,
        trend['buy'] / trend_total,
        trend['strongBuy'] / trend_total
    ]
    base_frames = stock_candles_by_date(ticker, date)
    return [list(x) + static_data for x in base_frames]

Go through the calendar and fetch all the historical data on this ticker that we have access to.
Let's start with 3 months.

In [124]:
def get_historical_data(ticker, days = 90):
    # if this historical pickle file exists, just return it
    historical_pickle_name = "{0}.technical.{1}.historical.df.pkl".format(ticker, time_period)
    if jar.pickle_exists(historical_pickle_name):
        return jar.read_pickle_dataframe(historical_pickle_name)
    # if it doesn't, build it
    historical_data = pandas.DataFrame()
    today = date.today()
    x_days_ago = today + timedelta(days = -1 * days)
    current_date = x_days_ago
    while current_date < today:
        # look for pickle file for this days data for this day's ticker
        pickle_name = "{0}-{1}.{2}.technical.df.pkl".format(ticker, current_date.strftime("%m-%d-%Y"), time_period)
        if jar.pickle_exists(pickle_name):
            data = jar.read_pickle_dataframe(pickle_name)
        else:
            data = pandas.DataFrame.from_records(
                construct_data_frames(ticker, current_date))
            # pickle this day's data to cut down on API requests
            jar.write_pickle_dataframe(pickle_name, data)
            time.sleep(2) # sleep for 2 seconds so we don't hit the API limit
        historical_data = historical_data.append(data)
        current_date = current_date + timedelta(days = 1)
    # label & type the data frame
    historical_data.columns = ['ts', 'o', 'l', 'h', 'c', 'v', 'e_a', 'e_e', 'r_ss', 'r_s', 'r_h', 'r_b', 'r_sb']
    historical_data['ts'] = pandas.to_datetime(historical_data['ts'], unit = 's')
    historical_data.index.name = 'ts'
    # pickle this historical data
    jar.write_pickle_dataframe(historical_pickle_name, historical_data)
    return historical_data

In [125]:
df = get_historical_data(ticker)

In [126]:
df.describe()

,o,l,h,c,v,e_a,e_e,r_ss,r_s,r_h,r_b,r_sb
count,1794.000000,1794.000000,1794.000000,1794.000000,1.794000e+03,1.794000e+03,1794.000000,1794.0,1794.0,1794.0,1794.0,1794.0
mean,128.691088,130.574125,130.510833,132.123825,1.072568e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
std,5.532129,5.827971,5.596367,5.705862,3.031395e+07,2.221065e-16,0.000000,0.0,0.0,0.0,0.0,0.0
min,117.570000,120.500000,118.899000,121.670000,5.493006e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
25%,123.449000,124.940000,126.600000,127.930000,8.693979e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
50%,128.500000,129.200000,130.890000,131.685000,9.839056e+07,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
75%,133.590000,135.580000,134.870000,136.310000,1.210473e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
max,141.370000,143.600000,143.160000,145.090000,1.925415e+08,-5.300000e-01,-0.857116,0.0,0.0,0.0,0.0,0.0
